## 2013 Club Championship Stats

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [7]:
stats = pd.read_html('../data/2013ClubChampStats.html')

In [8]:
stats_2013 = stats[0]
stats_2013.head()

,Team,#,Player,Goals,Assists,Ds,TOs
0,Bent,0,Brittany Kaplan,0,3,2,5
1,Bent,1,Lexi Zalk,4,1,4,0
2,Bent,2,Karen Chalif,3,1,0,6
3,Bent,3,Judy Jarvis,3,7,2,11
4,Bent,5,Katherine Cole,1,1,1,2


## 2014 Club Championship Stats

In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [101]:
def get_team_stat_urls(Championship_Page_URL):
    import requests
    from bs4 import BeautifulSoup
    
    r = requests.get(Championship_Page_URL) 
    b = BeautifulSoup(r.text, 'html.parser')

    step1 = [str(item).split('\n') for item in b.find_all('tr', style='border: 1px solid #eee;')]
    step2 = [item[1][13:-9] for item in step1]
    step3 = [item.split('">') for item in step2]
    step4 = [item[1].split(' (') for item in step3]
    
    dict_name = {}
    
    for item in range(16):
        dict_name[step4[item][0]] = step3[item][0] 
    
    return dict_name

In [102]:
teams_urls_2014 = get_team_stat_urls('http://play.usaultimate.org/events/USA-Ultimate-National-Championships/schedule/Women/Club-Women/')

In [103]:
teams_urls_2014

{'Brute Squad': '/events/teams/?EventTeamId=xp%2bejIJBOcfXXkdUBdhyKw7QdaKTsy2msZaYJsN0QFI%3d',
 'Capitals': '/events/teams/?EventTeamId=QMEULDssKTN4OuwrBMNq94efa7qxYCOB39XjXXvDBuY%3d',
 'Fury': '/events/teams/?EventTeamId=CBkbxJkqtPUazzlNKxrMYL476TAvWyT%2b%2bII9vIya7AM%3d',
 'Green Means Go': '/events/teams/?EventTeamId=ejfj3ptI1RiAMMNBDBMR%2bOWMPBwsN0Y86S%2fuCAYPw9U%3d',
 'Heist': '/events/teams/?EventTeamId=h%2bkaG2ZXhVd798X3lGV7PeEc5SUIuvMgQy9BNMokNsM%3d',
 'Molly Brown': '/events/teams/?EventTeamId=cKqY9nt%2bZuiYvmdlJ2MwdG0cK1L9yUN%2bWnowMqR1zKg%3d',
 'Nemesis': '/events/teams/?EventTeamId=mIbs2PBWcyioQl2JAuLt1FjcCr3eIAK7iEJX3hRzIO8%3d',
 'Nightlock': '/events/teams/?EventTeamId=KWGXFcYyimfQtEQ%2fDYoEE7kZ1HzzEC0yN7KIfYlhSWg%3d',
 'Ozone': '/events/teams/?EventTeamId=01oilPzTzWp6xnsGkQ0uJtZl64NEdUOHJ0wQZrvMnNY%3d',
 'Riot': '/events/teams/?EventTeamId=ddLOZCRcEzHqNrt1jat3nz4wNJ%2f7YNKK5OZQonhRuaQ%3d',
 'Scandal': '/events/teams/?EventTeamId=m%2f3%2f0BtHZz5iI3aEgO1fqtpMAlDOwL7dnQ9uEQ

In [105]:
# explore existing columns

r = requests.get('http://play.usaultimate.org' + teams_urls_2014['Traffic']) #any team name will do
team_page = BeautifulSoup(r.text, 'html.parser')

In [106]:
columns_2014 = [str(item)[15:-5] for item in team_page.find('div', id='roster_list').find_all('th', span='col')]

In [107]:
columns_2014

['No.',
 'Name',
 'Position',
 'Year',
 'Height',
 'Goals',
 'Assists',
 'Ds',
 'Turns']

In [108]:
def get_stats_page(team_name, dict_name):
    # get html of team stats page
    r = requests.get('http://play.usaultimate.org' + dict_name[team_name])
    team_page = BeautifulSoup(r.text, 'html.parser')
    return team_page.find('div', id='roster_list')

In [112]:
# get_stats_page('Brute Squad', teams_urls_2014)

In [ ]:
# good so far (12/28 9:30pm)

In [86]:
def organize_stats(team_name):
    '''
    team_name is a string
    creates a list of values per stat category
    '''
    
    List2 = []
    
    numbers = []
    names = []
    position = []
    year = []
    height = []
    goals = []
    assists = []
    ds = []
    turns = []
    Team = []
    
    # create list of events
    get_stats_page(team_name).find_all('td')
    List1 = [str(item)[4:-5] for item in get_stats_page(team_name).find_all('td')]
    
    # strip tags
    count = 0

    while count < len(List1):
        List2.append(List1[count:count+9])
        count +=9
    
    # add stripped events to list
    for item in range(len(List2)):
        Team.append(team_name)
        numbers.append(List2[item][0])
        names.append(List2[item][1])
        position.append(List2[item][2])
        year.append(List2[item][3])
        height.append(List2[item][4])
        goals.append(List2[item][5])
        assists.append(List2[item][6])
        ds.append(List2[item][7])
        turns.append(List2[item][8])
    

def add_stats(df):
    '''
    adds stats to df
    '''
        
    df['No.']=numbers
    df['Name']=names
    df['Team']=Team
    df['Position']=position
    df['Year']=year
    df['Height']=height
    df['Goals']=goals
    df['Assists']=assists
    df['Ds']=ds
    df['Turns']=turns

def get_ints(df):
    '''
    converts goals, assists, ds and turns into ints
    '''
    
    df['Goals']=df['Goals'].astype('int')
    df['Assists']=df['Assists'].astype('int')
    df['Ds']= df['Ds'].astype('int')
    df['Turns']=df['Turns'].astype('int')

In [89]:
def build_stats_df(df, stats_year):
    '''
    df name should denote year (e.g. stats_2014)
    stat year should denote year (e.g. teams_2014)
    '''
    
    df = pd.DataFrame()
    
    for team in stats_year:
        organize_stats(str(team))
    
    add_stats(df)
    
    get_ints(df)

In [90]:
build_stats_df(stats_2014, teams_2014)

NameError: name 'stats_2014' is not defined

In [352]:
stats_2014.dtypes

No.         object
Name        object
Team        object
Position    object
Year        object
Height      object
Goals        int64
Assists      int64
Ds           int64
Turns        int64
dtype: object

In [358]:
stats_2014.sort_values('Goals', ascending=False) # sort not working for goals/assists

,No.,Name,Team,Position,Year,Height,Goals,Assists,Ds,Turns
213,21,KRISTINA SNODGRASS,Molly Brown,,,"5'11""",22,2,0,0
338,37,SANDRA JORGENSEN,Scandal,,,,21,4,0,0
198,57,ABBEY HEWITT,Ozone,,,,16,3,0,0
375,12,KATIE RYAN,Green Means Go,Deep,,,15,4,0,0
351,07,KELSEY VIARS,Tabby Rosa,,,"5'5""",14,3,0,0
263,33,LAURA MASON,Traffic,,,"5'8""",14,7,0,0
87,16,SARAH LEVINN,Showdown,,,,13,1,0,0
240,31,SARA SCOTT,Nemesis,,,,13,17,0,0
123,51,CLAIRE DESMOND,Fury,,,,12,9,0,0
128,03,LIZA MINOR,Heist,,,,12,9,0,0


## 2015 Club Championship Stats

In [3]:
import requests
from bs4 import BeautifulSoup

r = requests.get('http://play.usaultimate.org/events/USA-Ultimate-National-Championships-2015/schedule/Women/Club-Women/') 
b = BeautifulSoup(r.text, 'html.parser')

step1 = [str(item).split('\n') for item in b.find_all('tr', style='border: 1px solid #eee;')]
step2 = [item[1][13:-9] for item in step1]
step3 = [item.split('">') for item in step2]
step3
step4 = [item[1].split(' (') for item in step3]

In [35]:
def team_stat_url(url):
    import requests
    from bs4 import BeautifulSoup
    
    r = requests.get(url) 
    b = BeautifulSoup(r.text, 'html.parser')

    step1 = [str(item).split('\n') for item in b.find_all('tr', style='border: 1px solid #eee;')]
    step2 = [item[1][13:-9] for item in step1]
    step3 = [item.split('">') for item in step2]
    step4 = [item[1].split(' (') for item in step3]
    
    dict_name = {}
    
    for item in range(16):
        dict_name[step4[item][0]] = step3[item][0] 
    
    return dict_name

In [36]:
teams_2015 = team_stat_url('http://play.usaultimate.org/events/USA-Ultimate-National-Championships-2015/schedule/Women/Club-Women/')

In [37]:
teams_2015

{'BENT': '/events/teams/?EventTeamId=tl1rkVLjdczXVbOWzjG8T6fVjubOyhxGoQqAHV0da4g%3d',
 'Brute Squad': '/events/teams/?EventTeamId=c%2bbyh9S9ANQUYGFX55jj2huoFtowEZg%2fAyga4pprMQ0%3d',
 'Capitals': '/events/teams/?EventTeamId=x4u7Gvp5rOjImw437isTD%2fs%2bQ06953u7JY%2f95O2ZmYY%3d',
 'Fury': '/events/teams/?EventTeamId=32oHgC3wTopGTV3EhDSbH36OX0nyemEg%2bP5KQfBRE1g%3d',
 'Heist': '/events/teams/?EventTeamId=CpQQbzcjJ0UXcqhrXjm3d8zVp0%2fIhQBVr%2fCVAX8yEAY%3d',
 'Iris': '/events/teams/?EventTeamId=Wy3HoIVoyiewP6W0uRSYiIJo8RJY0m3zrLvGpRwtEwI%3d',
 'Molly Brown': '/events/teams/?EventTeamId=RbDtI%2bYRGJIFXwMyY2GXHZNkAP4TG3e5cYp%2bwYagX5s%3d',
 'Nemesis': '/events/teams/?EventTeamId=crhCXpLrJODvUJL6ppk9K1V8i2mXHNLATKv1ZOibPUc%3d',
 'Nightlock': '/events/teams/?EventTeamId=EfdvGasY6mEwfXfzbwlKQKOo3de2IF4mbYrPHnmtd2Q%3d',
 'Ozone': '/events/teams/?EventTeamId=iAImqQuifQ77w%2bC0P2niH56jBIDV%2fpmjWfCpaiDP%2brQ%3d',
 'Phoenix': '/events/teams/?EventTeamId=CB%2br3JHfJtcIvZONww0nNVHSZZMq9jH%2frilvqOvHLy

In [5]:
teams_2015

{'BENT': '/events/teams/?EventTeamId=tl1rkVLjdczXVbOWzjG8T6fVjubOyhxGoQqAHV0da4g%3d',
 'Brute Squad': '/events/teams/?EventTeamId=c%2bbyh9S9ANQUYGFX55jj2huoFtowEZg%2fAyga4pprMQ0%3d',
 'Capitals': '/events/teams/?EventTeamId=x4u7Gvp5rOjImw437isTD%2fs%2bQ06953u7JY%2f95O2ZmYY%3d',
 'Fury': '/events/teams/?EventTeamId=32oHgC3wTopGTV3EhDSbH36OX0nyemEg%2bP5KQfBRE1g%3d',
 'Heist': '/events/teams/?EventTeamId=CpQQbzcjJ0UXcqhrXjm3d8zVp0%2fIhQBVr%2fCVAX8yEAY%3d',
 'Iris': '/events/teams/?EventTeamId=Wy3HoIVoyiewP6W0uRSYiIJo8RJY0m3zrLvGpRwtEwI%3d',
 'Molly Brown': '/events/teams/?EventTeamId=RbDtI%2bYRGJIFXwMyY2GXHZNkAP4TG3e5cYp%2bwYagX5s%3d',
 'Nemesis': '/events/teams/?EventTeamId=crhCXpLrJODvUJL6ppk9K1V8i2mXHNLATKv1ZOibPUc%3d',
 'Nightlock': '/events/teams/?EventTeamId=EfdvGasY6mEwfXfzbwlKQKOo3de2IF4mbYrPHnmtd2Q%3d',
 'Ozone': '/events/teams/?EventTeamId=iAImqQuifQ77w%2bC0P2niH56jBIDV%2fpmjWfCpaiDP%2brQ%3d',
 'Phoenix': '/events/teams/?EventTeamId=CB%2br3JHfJtcIvZONww0nNVHSZZMq9jH%2frilvqOvHLy

NameError: name 't' is not defined